In [1]:
#libraries
import pandas as pd

##RESULT Headers##
columns=['Overnight Percentage time in hyperglycemia (CGM > 180 mg/dL)',
       'Overnight percentage of time in hyperglycemia critical (CGM > 250 mg/dL)',
       'Overnight percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL)',
       'Overnight percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL)',
       'Overnight percentage time in hypoglycemia level 1 (CGM < 70 mg/dL)',
       'Overnight percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)',
       'Daytime Percentage time in hyperglycemia (CGM > 180 mg/dL)',
       'Daytime percentage of time in hyperglycemia critical (CGM > 250 mg/dL)',
       'Daytime percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL)',
       'Daytime percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL)',
       'Daytime percentage time in hypoglycemia level 1 (CGM < 70 mg/dL)',
       'Daytime percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)',
       'Whole Day Percentage time in hyperglycemia (CGM > 180 mg/dL)',
       'Whole day percentage of time in hyperglycemia critical (CGM > 250 mg/dL)',
       'Whole day percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL)',
       'Whole day percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL)',
       'Whole day percentage time in hypoglycemia level 1 (CGM < 70 mg/dL)',
       'Whole Day percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)']

index=['Manual Mode', 'Auto Mode']
df_output=pd.DataFrame(columns=columns,index=index)

#### Insulin Data - Preprocessing ####
df_insulin=pd.read_csv("InsulinData.csv")
#Attribute transformation - creating timestamp attribute
df_insulin["Timestamp"]=pd.to_datetime(df_insulin["Date"]+" "+df_insulin["Time"])
# AUTO mode starting point index
auto_index=df_insulin.loc[df_insulin["Alarm"]=="AUTO MODE ACTIVE PLGM OFF"].index[-1]
#Data subset selection
df_autoInsulin=df_insulin.iloc[:auto_index+1].iloc[::-1].reset_index(drop=True)
#Dimensionality reduction
df_autoInsulin=df_autoInsulin[["Timestamp","Alarm"]]
# AUTO mode start time
autoStartTime=df_autoInsulin.iloc[0,0]
#Handling missing values for Insulin data
df_autoInsulin.dropna(inplace=True)
    

#### CGM Data - Preprocessing ####
df_cgm=pd.read_csv("CGMData.csv")
#Attribute transformation - creating timestamp attribute
df_cgm["Timestamp"]=pd.to_datetime(df_cgm["Date"]+" "+df_cgm["Time"])
#Converting to Date Time type
df_cgm["Date"] = pd.to_datetime(df_cgm["Date"])
df_cgm["Time"] = pd.to_datetime(df_cgm["Time"]).dt.time
#Divide CGM data into auto & manual mode data using this index
autoCGM_index=df_cgm[df_cgm["Timestamp"]>autoStartTime].index[-1]
#Manual mode CGM data
df_manCGM=df_cgm.iloc[autoCGM_index+1:].iloc[::-1].reset_index(drop=True)
#Auto mode CGM data
df_autoCGM=df_cgm.iloc[:autoCGM_index+1].iloc[::-1].reset_index(drop=True)
#Dimensionality reduction
df_manCGM=df_manCGM[["Timestamp","Date","Time","Sensor Glucose (mg/dL)"]]
df_autoCGM=df_autoCGM[["Timestamp","Date","Time","Sensor Glucose (mg/dL)"]]
#Handling missing values for CGM
#step1: technique - deletion
df_manCGM=df_manCGM.dropna()
df_autoCGM=df_autoCGM.dropna()
# #step 2: technique - delete days without 288 datapoints using the dates_x set
# df_filtered_man=df_manCGM.groupby("Date").filter(lambda x: len(x)==288)
# df_manCGM_day=df_manCGM[df_manCGM["Date"].isin(df_filtered_man["Date"])]
# df_filtered_auto=df_autoCGM.groupby("Date").filter(lambda x: len(x)==288)
# df_autoCGM_day=df_autoCGM[df_autoCGM["Date"].isin(df_filtered_auto["Date"])]
# #step2: technique - delete days with na values 
# df_filtered1_man=df_manCGM[df_manCGM["Sensor Glucose (mg/dL)"].isna()]
# df_manCGM=df_manCGM[~df_manCGM["Date"].isin(df_filtered1_man["Date"])]
# df_filtered1_auto=df_autoCGM[df_autoCGM["Sensor Glucose (mg/dL)"].isna()]
# df_autoCGM=df_autoCGM[~df_autoCGM["Date"].isin(df_filtered1_auto["Date"])] 
# #step 2: technique - interpolate days with na values
# df_manCGM['Sensor Glucose (mg/dL)'] = df_manCGM['Sensor Glucose (mg/dL)'].interpolate(method='linear')
# df_autoCGM['Sensor Glucose (mg/dL)'] = df_autoCGM['Sensor Glucose (mg/dL)'].interpolate(method='linear')
#step 2: technique - interpolate wrt time
df_manCGM.set_index("Timestamp",inplace=True)
df_manCGM['Sensor Glucose (mg/dL)']=df_manCGM['Sensor Glucose (mg/dL)'].interpolate(method='time')
df_autoCGM.set_index("Timestamp",inplace=True)
df_autoCGM['Sensor Glucose (mg/dL)']=df_autoCGM['Sensor Glucose (mg/dL)'].interpolate(method='time')

##Function to calculate output metrics and fill df_output##
#INPUT: df_input - filtered dataframe, col_start - starting column in the output dataframe
#,auto - flag for auto or manual mode, whole_day_segment - flag if metrics to be taken for the whole day
#OUTPUT: populates the output data frame
def calculate_metrics(df_input,col_start,auto,whole_day_segment=False): 
    # calculation for each group of data
    def custom_agg(group):
        return pd.Series({
            'group length':len(group),
            'hyperglycemia':(len(group[group["Sensor Glucose (mg/dL)"]>180.0])/288)*100,
            'hyperglycemia critical': (len(group[group["Sensor Glucose (mg/dL)"]>250.0])/288)*100,
            'range': (len(group[(group["Sensor Glucose (mg/dL)"]>=70.0) & (group["Sensor Glucose (mg/dL)"]<=180)])/288)*100,
            'range secondary': (len(group[(group["Sensor Glucose (mg/dL)"]>=70.0) & (group["Sensor Glucose (mg/dL)"]<=150)])/288)*100,
            'hypoglycemia level1': (len(group[group["Sensor Glucose (mg/dL)"]<70.0])/288)*100,
            'hypoglycemia level2': (len(group[group["Sensor Glucose (mg/dL)"]<54.0])/288)*100
        }) 
    auto = 1 if auto else 0
    df_dailyPercent=df_input.groupby("Date").apply(custom_agg).reset_index()
    #daily percent average
    df_output.iloc[auto,col_start]=df_dailyPercent["hyperglycemia"].mean()
    df_output.iloc[auto,col_start+1]=df_dailyPercent["hyperglycemia critical"].mean()
    df_output.iloc[auto,col_start+2]=df_dailyPercent["range"].mean()
    df_output.iloc[auto,col_start+3]=df_dailyPercent["range secondary"].mean()
    df_output.iloc[auto,col_start+4]=df_dailyPercent["hypoglycemia level1"].mean()
    df_output.iloc[auto,col_start+5]=df_dailyPercent["hypoglycemia level2"].mean()
    return


#Case A - Manual mode
#Segment - Overnight
df_overnight=df_manCGM[(df_manCGM["Time"]<=pd.to_datetime('06:00:00').time()) & (df_manCGM["Time"]>=pd.to_datetime('00:00:00').time())]
calculate_metrics(df_overnight,df_output.columns.get_loc("Overnight Percentage time in hyperglycemia (CGM > 180 mg/dL)"),False)
#Segment - Daytime
df_day=df_manCGM[(df_manCGM["Time"]>=pd.to_datetime('06:00:00').time()) & (df_manCGM["Time"]<=pd.to_datetime('23:59:59').time())]
calculate_metrics(df_day,df_output.columns.get_loc("Daytime Percentage time in hyperglycemia (CGM > 180 mg/dL)"),False)
#Segment - 24 hours (expected 288 samples)
calculate_metrics(df_manCGM,df_output.columns.get_loc("Whole Day Percentage time in hyperglycemia (CGM > 180 mg/dL)"),False,True)

#Case B - Auto mode
#Segment - Overnight
df_overnight=df_autoCGM[(df_autoCGM["Time"]<=pd.to_datetime('06:00:00').time()) & (df_autoCGM["Time"]>=pd.to_datetime('00:00:00').time())]
calculate_metrics(df_overnight,df_output.columns.get_loc("Overnight Percentage time in hyperglycemia (CGM > 180 mg/dL)"),True)
#Segment - Daytime
df_day=df_autoCGM[(df_autoCGM["Time"]>=pd.to_datetime('06:00:00').time()) & (df_autoCGM["Time"]<=pd.to_datetime('23:59:59').time())]
calculate_metrics(df_day,df_output.columns.get_loc("Daytime Percentage time in hyperglycemia (CGM > 180 mg/dL)"),True)
#Segment - 24 hours (expected 288 samples)
calculate_metrics(df_autoCGM,df_output.columns.get_loc("Whole Day Percentage time in hyperglycemia (CGM > 180 mg/dL)"),True,True)

# Write output to Result.csv
df_output.to_csv('Result.csv',index=False,header=False)

/Users/fatimabasheer/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
<ipython-input-1-485cafea15ff>:28: DtypeWarning: Columns (13,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df_insulin=pd.read_csv("InsulinData.csv")
<ipython-input-1-485cafea15ff>:44: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cgm=pd.read_csv("CGMData.csv")
<ipython-input-1-485cafea15ff>:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_cgm["Time"] = pd.to_datetime(df_cgm["Time"]).dt.time


In [2]:
df_output.head()

,Overnight Percentage time in hyperglycemia (CGM > 180 mg/dL),Overnight percentage of time in hyperglycemia critical (CGM > 250 mg/dL),Overnight percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL),Overnight percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL),Overnight percentage time in hypoglycemia level 1 (CGM < 70 mg/dL),Overnight percentage time in hypoglycemia level 2 (CGM < 54 mg/dL),Daytime Percentage time in hyperglycemia (CGM > 180 mg/dL),Daytime percentage of time in hyperglycemia critical (CGM > 250 mg/dL),Daytime percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL),Daytime percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL),Daytime percentage time in hypoglycemia level 1 (CGM < 70 mg/dL),Daytime percentage time in hypoglycemia level 2 (CGM < 54 mg/dL),Whole Day Percentage time in hyperglycemia (CGM > 180 mg/dL),Whole day percentage of time in hyperglycemia critical (CGM > 250 mg/dL),Whole day percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL),Whole day percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL),Whole day percentage time in hypoglycemia level 1 (CGM < 70 mg/dL),Whole Day percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)
Manual Mode,4.837963,1.018519,18.240741,13.402778,0.671296,0.0,22.829861,8.463542,34.830729,27.105035,4.578993,1.801215,27.365451,9.418403,51.931424,39.670139,5.208333,1.801215
Auto Mode,2.600265,0.405743,20.265684,17.681024,0.630072,0.152154,19.535108,4.560185,45.115741,33.800154,3.31983,0.983796,22.106481,4.96142,65.15625,51.284722,3.942901,1.134259


In [3]:
df_output.head()

,Overnight Percentage time in hyperglycemia (CGM > 180 mg/dL),Overnight percentage of time in hyperglycemia critical (CGM > 250 mg/dL),Overnight percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL),Overnight percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL),Overnight percentage time in hypoglycemia level 1 (CGM < 70 mg/dL),Overnight percentage time in hypoglycemia level 2 (CGM < 54 mg/dL),Daytime Percentage time in hyperglycemia (CGM > 180 mg/dL),Daytime percentage of time in hyperglycemia critical (CGM > 250 mg/dL),Daytime percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL),Daytime percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL),Daytime percentage time in hypoglycemia level 1 (CGM < 70 mg/dL),Daytime percentage time in hypoglycemia level 2 (CGM < 54 mg/dL),Whole Day Percentage time in hyperglycemia (CGM > 180 mg/dL),Whole day percentage of time in hyperglycemia critical (CGM > 250 mg/dL),Whole day percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL),Whole day percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL),Whole day percentage time in hypoglycemia level 1 (CGM < 70 mg/dL),Whole Day percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)
Manual Mode,4.837963,1.018519,18.240741,13.402778,0.671296,0.0,22.829861,8.463542,34.830729,27.105035,4.578993,1.801215,27.365451,9.418403,51.931424,39.670139,5.208333,1.801215
Auto Mode,2.600265,0.405743,20.265684,17.681024,0.630072,0.152154,19.535108,4.560185,45.115741,33.800154,3.31983,0.983796,22.106481,4.96142,65.15625,51.284722,3.942901,1.134259


In [4]:
df_manCGM

,Date,Time,Sensor Glucose (mg/dL)
Timestamp,,,
2017-07-25 12:08:54,2017-07-25,12:08:54,314.0
2017-07-25 12:13:54,2017-07-25,12:13:54,310.0
2017-07-25 12:18:54,2017-07-25,12:18:54,309.0
2017-07-25 12:23:54,2017-07-25,12:23:54,311.0
2017-07-25 12:28:54,2017-07-25,12:28:54,311.0
...,...,...,...
2017-08-09 07:45:05,2017-08-09,07:45:05,160.0
2017-08-09 07:50:05,2017-08-09,07:50:05,157.0
2017-08-09 07:55:05,2017-08-09,07:55:05,158.0
